In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 38.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re

#from nltk.tokenize import word_tokenize
from pythainlp.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer

# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam

import seaborn as sn
import matplotlib.pyplot as plt

import pickle as p
import plotly
import plotly.graph_objs as go

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('train_message.csv')
df = df.drop_duplicates()

messages = list(df['message'])
category = list(df['category'])

df.head()

,category,message
0,positive,เจิมให้
1,negative,เพื่อนสนิทกูแม่งได้แฟนที่มีปัญหาครอบครัวหนักมา...
2,negative,รําคาญนิสัยเพื่อนหวะเเม่งด่าเพื่อนอีกคนว่าเวลา...
3,negative,กุมีประโยชน์แค่เอาไว้ถามงานมั้งไอ้เหี้ย บางอัน...
4,positive,กอด ไม่ต่างกัน คำถามที่บางที่เสิจอากู๋ก็เจอยั...


In [ ]:
def cleaning(sentences):
  words = []
  temp = []
  for s in sentences:
    clean = re.sub(r'[^ก-๙]', "", s)
    w = word_tokenize(clean)
    temp.append([i.lower() for i in w])
    words.append(' '.join(w).lower())

  return words, temp

In [ ]:
cleaned_words, temp = cleaning(messages)
print(len(cleaned_words))
#print(cleaned_words[:5])
#print(temp)

401


In [ ]:
def create_tokenizer(words, filters = ''):
    token = Tokenizer(filters=filters)
    token.fit_on_texts(words)
    return token

In [ ]:
train_word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(train_word_tokenizer.word_index) + 1

train_word_tokenizer.word_index

{'ไม่': 1,
 'กู': 2,
 'ก็': 3,
 'มัน': 4,
 'มึง': 5,
 'ไป': 6,
 'เพื่อน': 7,
 'ที่': 8,
 'จะ': 9,
 'มา': 10,
 'กุ': 11,
 'แต่': 12,
 'มี': 13,
 'ได้': 14,
 'ว่า': 15,
 'คน': 16,
 'เลย': 17,
 'ๆ': 18,
 'แล้ว': 19,
 'กับ': 20,
 'เป็น': 21,
 'ให้': 22,
 'ทำ': 23,
 'กัน': 24,
 'แม่': 25,
 'อะไร': 26,
 'นะ': 27,
 'ง': 28,
 'อยาก': 29,
 'เพราะ': 30,
 'ถ้า': 31,
 'อยู่': 32,
 'แบบ': 33,
 'ใน': 34,
 'คือ': 35,
 'ต้อง': 36,
 'เรื่อง': 37,
 'คุย': 38,
 'เหี้ย': 39,
 'มาก': 40,
 'กลุ่ม': 41,
 'ดี': 42,
 'วะ': 43,
 'แค่': 44,
 'ของ': 45,
 'เคย': 46,
 'ตัวเอง': 47,
 'พอ': 48,
 'ชอบ': 49,
 'คิด': 50,
 'ยัง': 51,
 'พวก': 52,
 'คนอื่น': 53,
 'ละ': 54,
 'ด้วย': 55,
 'บอก': 56,
 'อีก': 57,
 'หรือ': 58,
 'นี่': 59,
 'ไร': 60,
 'อ่ะ': 61,
 'รู้': 62,
 'เวลา': 63,
 'กว่า': 64,
 'รู้สึก': 65,
 'พูด': 66,
 'นี้': 67,
 'งาน': 68,
 'หา': 69,
 'เหมือน': 70,
 'เอง': 71,
 'เอา': 72,
 'นั้น': 73,
 'เจอ': 74,
 'เขา': 75,
 'รอ': 76,
 'สนิท': 77,
 'โดน': 78,
 'บ้าง': 79,
 'ใคร': 80,
 'และ': 81,
 'นึง': 82,
 'จริงๆ': 

In [ ]:
def max_length(words):
    return(len(max(words, key = len)))

In [ ]:
max_length = max_length(temp)
max_length

339

In [ ]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [ ]:
encoded_doc = encoding_doc(train_word_tokenizer, cleaned_words)

print(cleaned_words[0])
print(encoded_doc[0])

เจิม ให้
[1356, 22]


In [ ]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)
print("Shape of padded docs = ",padded_doc.shape)

print(cleaned_words[0])
print(encoded_doc[0])
print(padded_doc[0])

Shape of padded docs =  (401, 339)
เจิม ให้
[1356, 22]
[1356   22    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  

In [ ]:
unique_category = list(set(category))
unique_category

['negative', 'positive']

In [ ]:
output_tokenizer = create_tokenizer(unique_category)

In [ ]:
encoded_output = encoding_doc(output_tokenizer, category)
print(category[:10])
print(encoded_output[:10])

['positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive']
[[2], [1], [1], [1], [2], [2], [2], [2], [2], [2]]


In [ ]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)
encoded_output.shape

(401, 1)

In [ ]:
def one_hot(encode):
  oh = OneHotEncoder(sparse_output = False)
  return(oh.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)
print(encoded_output[0])
print(output_one_hot[0])

[2]
[0. 1.]


In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.3, stratify=output_one_hot)

In [ ]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (280, 339) and train_Y = (280, 2)
Shape of val_X = (121, 339) and val_Y = (121, 2)


In [ ]:
num_classes = len(unique_category)
num_classes

2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import load_model

from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)

In [ ]:
def create_model(vocab_size, max_length):
  model = Sequential()

  model.add(Embedding(vocab_size, 128, input_length = max_length,  trainable = True))
  model.add(Bidirectional(GRU(128, activation = "relu"))) # activation = "relu"
  model.add(Dense(128, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation = "softmax"))

  return model

model = create_model(vocab_size, max_length)

In [ ]:
# หา learning rate ที่ได้ค่า loss น้อยที่สุด
EPOCHS = 20
BS = [18, 22, 28, 32, 34]
loss_history = {}
lrs = [0.001, 0.002, 0.003, 0.004, 0.005, 0.0001, 0.0002, 0.0003]
toplr = [0.005, 0.0004, 0.0005]
files = []
history_files = []

In [ ]:
# หา learning rate ที่ได้ค่า loss น้อยที่สุด
for i in BS:
  print('Batch size :', i)
  #print('Learning rate :', i)
  model = create_model(vocab_size, max_length)
  model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])
  filename = f'model.h5({i}, 0.0001)'
  files.append(filename)
  checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

  hist = model.fit(train_X, train_Y, epochs = 1, batch_size = i, validation_data = (val_X, val_Y), callbacks = [checkpoint])
  history_model_name = f'history_model({i}, 0.0001)'
  history_files.append(history_model_name)
  with open(history_model_name, 'wb') as file:
    p.dump(hist.history, file)

  with open(history_model_name, 'rb') as file:
      his = p.load(file)

  print('')
  print(f'Batch size : {i}\nAccuracy :', his['loss'])
  #print(f'Learning rate : {i}\nAccuracy :', his['loss'])
  loss_history[i] = his['loss'][-1]
  print('\n')
  print('\n')

Batch size : 18
16/16 [==============================] - ETA: 0s - loss: 0.7593 - accuracy: 0.4929
Epoch 1: val_loss improved from inf to 0.69308, saving model to model.h5(18, 0.0001)
16/16 [==============================] - 46s 2s/step - loss: 0.7593 - accuracy: 0.4929 - val_loss: 0.6931 - val_accuracy: 0.4628



Batch size : 18
Accuracy : [0.7592536211013794]




Batch size : 22


StagingError: ignored

In [ ]:
# หา learning rate ที่ได้ค่า loss น้อยที่สุด
print('Loss history :\n', loss_history, '\n')
print('Min loss', min(loss_history.values()))

print('at learning rate :', list(loss_history.keys())[list(loss_history.values()).index(min(loss_history.values()))])

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])
model.summary()

In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
hist = model.fit(train_X, train_Y, epochs = EPOCHS, batch_size = BS, validation_data = (val_X, val_Y), callbacks = [checkpoint])

In [ ]:
with open('history_model', 'wb') as file:
    p.dump(hist.history, file)

with open('history_model', 'rb') as file:
    his = p.load(file)

In [ ]:
from IPython.display import HTML

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

h1 = go.Scatter(y=his['loss'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="loss"
                   )
h2 = go.Scatter(y=his['val_loss'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1, filename="Intent Classification")
HTML(fig1.to_html())

In [ ]:
h1 = go.Scatter(y=his['accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="acc"
                   )
h2 = go.Scatter(y=his['val_accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1, filename="Intent Classification")
HTML(fig1.to_html())

In [ ]:
# loss, accuracy ของแต่ละ learning rate
for file in history_files:
  print(file)
  predict_model = load_model(str(file))
  predict_model.summary()
  score = predict_model.evaluate(val_X, val_Y, verbose=0)
  print('')
  print(toplr[files.index(file)])
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

In [ ]:
predict_model = load_model(filename)
predict_model.summary()

In [ ]:
score = predict_model.evaluate(val_X, val_Y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predicted_classes = np.argmax(predict_model.predict(val_X), axis=-1)
predicted_classes.shape

In [ ]:
#predicted_classes = predict_model.predict_classes(val_X)
predicted_classes = predict_model.predict(val_X)
predicted_classes.shape

In [ ]:
y_true = np.argmax(val_Y,axis = 1)
print(val_Y[0])
print(y_true[0])

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)
np.savetxt("confusion_matrix.csv", cm, delimiter=",")

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cm, range(2), range(2))
plt.figure(figsize=(20,14))
sn.set(font_scale=1.2) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 14}, fmt='g') # for num predict size

plt.show()

In [ ]:
label_dict = output_tokenizer.word_index

In [ ]:
label = [key for key, value in label_dict.items()]

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=label, digits=4))